In [1]:
import numpy as np
import pandas as pd
from testgen_base import Test

In [2]:
import re
import os
import subprocess
import os.path as op
from tqdm import tqdm

In [3]:
if not op.exists("tasks"):
    os.makedirs("tasks")

In [4]:
if not op.exists("pdfs"):
    os.makedirs("pdfs")

In [5]:
if not op.exists("tex"):
    os.makedirs("tex")

In [6]:
if not op.exists("answers"):
    os.makedirs("answers")

In [7]:
if not op.exists("variables"):
    os.makedirs("variables")

In [8]:
class Test2(Test):
    
    def __init__(self):
        super(Test2, self).__init__()
        self.import1 = "import numpy as np"
        self.seed = "np.random.seed(1337)"
        self.uniform = "np.random.uniform(low=LOW, high=HIGH, size=10)"
        self.normal = "np.random.normal(loc=HIGH, scale=LOW, size=10)"
        
    def init_variable(self, current_variables, is_list):
        if is_list:
            s = np.random.choice(self.variables)+" = "+self.make_random_list()
        else:
            s = np.random.choice(current_variables)+" = "+self.random_expression(2, 3)
        return(s)
    
    def make_lcomp(self, lists, functions):
        s = self.list_comprehension.replace("FUNCTION", np.random.choice(functions))
        s = s.replace("VARIABLE", np.random.choice(self.variables))
        s = s.replace("LIST", np.random.choice(lists))
        return(s)
    
    def change_variable(self, current_variables):
        s = np.random.choice(current_variables)+" = "+self.random_expression(2, 3)
        s += np.random.choice(self.arithmetic)
        s += np.random.choice(current_variables)
        return(s)
        
    def make_random_list(self):
        what = np.random.choice(["uniform", "normal"])
        s = self.uniform if what=="uniform" else self.normal
        bounds = sorted(np.random.choice(np.arange(10), size=2))
        s = s.replace("LOW", str(bounds[0])).replace(
            "HIGH", str(bounds[1])
        ).replace("SIZE", str(np.random.choice(np.arange(5, 15))))
        return(s)
    
    def make_function(self):
        f_name = "".join(np.random.choice(self.variables, size=10))
        current_variables = np.random.choice(self.variables, size=1)
        s = self.function_start.replace(
            "NAME", f_name
        ).replace("ARGUMENTS", current_variables[0])
        s += "    "
        s += self.change_variable(current_variables)+"\n"
        s += "    "+self.function_return.replace("NAME", current_variables[0])+"\n"
        return(s, f_name)
    
    def make_code(self, n):
        code = []
        current_variables = []
        current_lists = []
        current_functions = []
        block_count = 0
        prev = "start"
        code.append(self.import1+"\n"+self.seed+"\n\n")
        for a in range(n):
            if len(current_variables) == 0:
                current_variables.append(np.random.choice(self.variables))
            what = np.random.choice(
                [
                    "assignment", "function"
                ], p = [0.5, 0.5]
            )
            if what == "assignment":
                line = self.init_variable(
                    current_variables, np.random.choice([True, False])
                )
                current_variables.append(line[0])
                if "random" in line:
                    current_lists.append(line[0])
                elif line[0] in current_lists:
                    del current_lists[current_lists.index(line[0])]
                current_variables = list(set(current_variables))
                code.append(block_count*"    ")
                code.append(line+"\n")
            elif what == "function":
                fline, fname = self.make_function()
                code.append(fline)
                current_functions.append(fname)
            #code += "\n"
            prev = what
      #  print(a, what)
        if len(current_lists) == 0:
            line = self.init_variable(
                current_variables, False
            )
            current_variables.append(line[0])
            current_lists.append(line[0])
            current_variables = list(set(current_variables))
            code.append(block_count*"    ")
            code.append(line+"\n")
        if len(current_functions) == 0:
            current_functions.extend(["np.sqrt", "np.log", "np.exp"])
        line = "print(np.round("+self.make_lcomp(current_lists, current_functions)+", 1))"
        code.append(line)
        return(code, current_variables)

In [9]:
t = Test2()

In [10]:
N_tasks = 400

In [11]:
for a in tqdm(range(N_tasks)):
    fn = op.join("tasks", str(a)+".py")
    ans_fn = op.join("answers", str(a)+".txt")
    var_fn = op.join("variables", str(a)+".txt")
    code, vs = t.make_code(np.random.choice(np.arange(5,20)))
    with open(fn, "w") as oh:
        oh.write("".join(code))
    the_code = subprocess.Popen(
        "python "+fn, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True
    )
    stdout, stderr = the_code.communicate()
    with open(ans_fn, "w") as oh:
        if len(stdout) != 0:
            oh.write("correct\n")
            oh.write(stdout.decode())
        else:
            oh.write("error\n")
            oh.write(stderr.decode())
    with open(var_fn, "w") as oh:
        oh.write(",".join(vs))

100%|██████████| 400/400 [01:09<00:00,  6.42it/s]


In [12]:
def form_answers(n):
    ans_fn = op.join("answers", str(n)+".txt")
    with open(ans_fn, "r") as ih:
        u = ih.readlines()
        if u[0].startswith("correct"):
            last_line = "".join(u[1:]).replace("\n", "")+"\n"
        else:
            last_line = u[-1]
    return(last_line)

In [13]:
def form_incorrect_answers(n, corrects, chosen):
    var_fn = op.join("variables", str(n)+".txt")
    with open(var_fn, "r") as ih1:
        #chosen = np.random.choice(corrects)
        out = chosen
        variables_in_file = ih1.read().replace("\n", "").split(",")
        what = np.random.choice(["chosen", "random"])
        if what == "chosen":
            if "name" in out:
                if len(variables_in_file)>1:
                    rpl = np.random.choice(variables_in_file)
                    out = re.sub("name \'[a-z]\'", "name \'"+rpl+"\'", chosen)
        elif what == "random":
            out = t.make_random_list()
    return(chosen)

In [14]:
form_answers(0)

'[3.5 3.7 3.5 3.3 3.4 3.2 3.5 2.7 2.9 3.8]\n'

In [15]:
correct_answers = [form_answers(a) for a in np.arange(N_tasks)]

In [16]:
form_incorrect_answers(6, correct_answers, np.random.choice(correct_answers))

'[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]\n'

In [17]:
possible_incorrect_answers = list(np.unique(correct_answers))

In [18]:
len(correct_answers)

400

In [23]:
all_answers = []
for i,a in tqdm(enumerate(correct_answers)):
    current_answers = [a]
    three_choices = np.random.choice(possible_incorrect_answers, size=3, replace=False)
    incorrect1 = form_incorrect_answers(i, correct_answers, three_choices[0])
    incorrect2 = form_incorrect_answers(i, correct_answers, three_choices[1])
    incorrect3 = form_incorrect_answers(i, correct_answers, three_choices[2])
    current_answers.extend(
        [incorrect1, incorrect2, incorrect3]
    )
    all_answers.append(np.random.permutation(current_answers))

400it [00:00, 2386.38it/s]


In [24]:
all_answers[0]

array(['[195.2 213.3 227.6 105.8 272.6  84.  207.6 283.7 386.7 138.5]\n',
       '[3.5 3.7 3.5 3.3 3.4 3.2 3.5 2.7 2.9 3.8]\n',
       '[335. 335. 335. 335. 335. 335. 335. 335. 335. 335.]\n',
       '[290.  430.6 367.9 249.7 336.  257.3 379.7 490.3 252.2 536.3]\n'],
      dtype='<U62')

In [25]:
variant = np.random.choice(np.arange(N_tasks), 10)

In [26]:
variant

array([175,  41,   3,  43, 215, 268, 190, 250,  34, 199])

In [27]:
from copy import deepcopy

In [28]:
all_answers[0]

array(['[195.2 213.3 227.6 105.8 272.6  84.  207.6 283.7 386.7 138.5]\n',
       '[3.5 3.7 3.5 3.3 3.4 3.2 3.5 2.7 2.9 3.8]\n',
       '[335. 335. 335. 335. 335. 335. 335. 335. 335. 335.]\n',
       '[290.  430.6 367.9 249.7 336.  257.3 379.7 490.3 252.2 536.3]\n'],
      dtype='<U62')

In [29]:
form = """
Вопрос №Q0, вариант VARIANT

LISTING0

1. A01
2. A02
3. A03
4. A04


Вопрос №Q1, вариант VARIANT

LISTING1

1. A11
2. A12
3. A13
4. A14


Вопрос №Q2, вариант VARIANT

LISTING2

1. A21
2. A22
3. A23
4. A24


Вопрос №Q3, вариант VARIANT

LISTING3

1. A31
2. A32
3. A33
4. A34


Вопрос №Q4, вариант VARIANT

LISTING4

1. A41
2. A42
3. A43
4. A44


Вопрос №Q5, вариант VARIANT

LISTING5

1. A51
2. A52
3. A53
4. A54


Вопрос №Q6, вариант VARIANT

LISTING6

1. A61
2. A62
3. A63
4. A64


Вопрос №Q7, вариант VARIANT

LISTING7

1. A71
2. A72
3. A73
4. A74


Вопрос №Q8, вариант VARIANT

LISTING8

1. A81
2. A82
3. A83
4. A84


Вопрос №Q9, вариант VARIANT

LISTING9

1. A91
2. A92
3. A93
4. A94

"""

In [30]:
def get_variant():
    variant = np.random.choice(np.arange(N_tasks), 10)
    current_form = deepcopy(form)
    for i,a in enumerate(variant):
        anssss = all_answers[a]
        question_number = "Q"+str(i)
        current_form = current_form.replace(question_number, str(a))
        answer_number_base = "A"+str(i)
        for j,b in enumerate(anssss):
            current_form = current_form.replace(answer_number_base+str(j+1), b)
        filename = op.join("tasks", str(a)+".py")
        with open(filename, "r") as ih:
            u = ih.read()
        current_form = current_form.replace("LISTING"+str(i), u)
    return(current_form)

In [31]:
for a in tqdm(range(30)):
    with open(op.join("tex/", str(a)+".txt"), "w") as oh:
        cf = get_variant()
        oh.write(cf.replace("VARIANT", str(a)))

100%|██████████| 30/30 [00:00<00:00, 636.28it/s]
